# Chapter 14: Nullable Types

---

## 14.1 Understanding `null` and `undefined`

TypeScript distinguishes between two absence-of-value types: `null` (intentional absence) and `undefined` (uninitialized or missing). Understanding their differences and handling them safely is crucial for writing robust TypeScript code.

### 14.1.1 `null` Type

`null` represents the intentional absence of any object value. It's a deliberate assignment indicating "no value here."

**Working with `null`:**

```typescript
// Explicit null type
function findUser(id: number): User | null {
  if (id <= 0) {
    return null; // Intentionally returning no user
  }
  return { id, name: "User " + id };
}

interface User {
  id: number;
  name: string;
}

const user = findUser(1);
// TypeScript knows user is User | null

if (user === null) {
  console.log("User not found");
} else {
  console.log(user.name); // TypeScript knows user is User here
}

// Null in object properties
interface Product {
  name: string;
  description: string | null; // Description might be intentionally absent
  discontinuedAt: Date | null; // Null if not discontinued
}

const product: Product = {
  name: "Laptop",
  description: null, // Intentionally no description
  discontinuedAt: null // Still active
};

// Must check for null before using
if (product.description !== null) {
  console.log(product.description.toUpperCase());
}
```

### 14.1.2 `undefined` Type

`undefined` indicates that a variable has been declared but not assigned a value, or that a property doesn't exist on an object.

**Working with `undefined`:**

```typescript
// Undefined from missing return
function getConfig(): string | undefined {
  if (Math.random() > 0.5) {
    return "config";
  }
  // Implicitly returns undefined
}

const config = getConfig();
// config is string | undefined

// Undefined for optional parameters
function greet(name?: string): string {
  if (name === undefined) {
    return "Hello, stranger!";
  }
  return `Hello, ${name}!`;
}

console.log(greet()); // "Hello, stranger!"
console.log(greet("John")); // "Hello, John!"

// Undefined in objects
interface Settings {
  theme: string;
  language?: string; // Optional - may be undefined
  notifications: boolean | undefined; // Explicitly undefined allowed
}

const settings: Settings = {
  theme: "dark",
  notifications: undefined
};

// Accessing optional property
console.log(settings.language); // undefined (not set)
console.log(settings.theme); // "dark"

// Optional chaining handles undefined gracefully
console.log(settings.language?.toUpperCase()); // undefined (no error)
```

### 14.1.3 Differences Between `null` and `undefined`

**Comparison Table:**

```
┌─────────────────────────────────────────────────────────────────────┐
│                    null vs undefined Comparison                        │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   Feature              │ null                    │ undefined       │
│  ──────────────────────┼─────────────────────────┼─────────────────│
│   Meaning              │ Intentional absence     │ Not initialized │
│   typeof               │ "object" (JS quirk)     │ "undefined"     │
│   JSON.stringify       │ preserved               │ omitted         │
│   Default parameter    │ treated as value        │ triggers default│
│   Optional chaining    │ stops at null           │ stops at undef  │
│   Use case             │ "No value exists"       │ "Not set yet"   │
│                                                                     │
│   Examples:                                                         │
│   ┌──────────────────────────────────────────────────────────────┐ │
│   │ let a = null;     // Explicitly nothing                     │ │
│   │ let b;            // Implicitly undefined                   │ │
│   │ let c = undefined; // Explicitly not set                  │ │
│   │                                                                │ │
│   │ // API design:                                                  │ │
│   │ user.spouse: User | null      // User has no spouse           │ │
│   │ user.middleName?: string      // User might not have one      │ │
│   └──────────────────────────────────────────────────────────────┘ │
│                                                                     │
│   Recommendation:                                                   │
│   • Use undefined for optional properties and parameters            │
│   • Use null for intentional absence (API responses, databases)     │
│   • Enable strictNullChecks to catch errors                          │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

**Practical Differences:**

```typescript
// typeof behavior
console.log(typeof null);      // "object" (JavaScript bug preserved)
console.log(typeof undefined); // "undefined"

// Equality checks
console.log(null == undefined);  // true (loose equality)
console.log(null === undefined); // false (strict equality)

// Default parameters
function withDefault(value = "default") {
  console.log(value);
}

withDefault();         // "default" (undefined triggers default)
withDefault(undefined); // "default" (undefined triggers default)
withDefault(null);      // null (null does NOT trigger default!)

// JSON serialization
const obj = {
  a: null,
  b: undefined
};

console.log(JSON.stringify(obj)); // {"a":null} (undefined omitted)

// Optional properties vs nullable
interface Person {
  nickname: string | null;  // Must be present, can be null
  middleName?: string;     // Can be omitted (undefined)
}

const person: Person = {
  nickname: null // Must include, even if null
  // middleName omitted - OK
};
```

---

## 14.2 Strict Null Checks

The `strictNullChecks` compiler option is one of TypeScript's most important features for catching null/undefined errors at compile time.

### 14.2.1 `strictNullChecks` Option

When enabled, `null` and `undefined` are treated as distinct types that must be explicitly handled.

**With strictNullChecks Enabled:**

```typescript
// tsconfig.json: "strictNullChecks": true

function getLength(str: string | null): number {
  // Without strictNullChecks, this would compile but crash at runtime
  // return str.length; // ❌ Error: Object is possibly 'null'
  
  // Must handle null case
  if (str === null) {
    return 0;
  }
  
  // Now TypeScript knows str is string
  return str.length;
}

// Nullable properties
interface User {
  name: string;
  email: string | null;
}

function sendEmail(user: User): void {
  // user.email might be null
  // console.log(user.email.toLowerCase()); // ❌ Error
  
  if (user.email !== null) {
    console.log(user.email.toLowerCase()); // ✅ OK
  }
}

// Array access returns union type
const items: string[] = ["a", "b", "c"];
const item = items[10]; // Type is string | undefined (might be out of bounds)

// Must check
if (item !== undefined) {
  console.log(item.toUpperCase());
}
```

**Without strictNullChecks (Legacy Behavior):**

```typescript
// "strictNullChecks": false

function dangerous(str: string): number {
  return str.length; // Compiles, but might crash if null passed
}

// null assignable to any type
let name: string = null; // No error!
let age: number = undefined; // No error!

// This is why strictNullChecks is essential for type safety
```

### 14.2.2 Impact on Code

**Code Changes Required:**

```typescript
// Before strictNullChecks (implicitly unsafe)
function processUser(user: User) {
  console.log(user.email.toLowerCase()); // Might crash
}

// After enabling strictNullChecks
function processUserSafe(user: User) {
  if (user.email === null) {
    console.log("No email");
    return;
  }
  
  // TypeScript knows email is string here
  console.log(user.email.toLowerCase());
}

// Handling optional properties
interface Config {
  apiUrl: string;
  timeout?: number;
}

function initialize(config: Config) {
  // config.timeout is number | undefined
  
  // Option 1: Check for undefined
  if (config.timeout !== undefined) {
    console.log(`Timeout: ${config.timeout}ms`);
  }
  
  // Option 2: Provide default
  const timeout = config.timeout ?? 5000;
  
  // Option 3: Use non-null assertion (if you're certain)
  // const timeout = config.timeout!; // Avoid if possible
}

// Function return types must be explicit
function findItem(id: number): Item | undefined {
  // Must return undefined explicitly or implicitly
  if (id < 0) return undefined;
  return { id };
}

interface Item {
  id: number;
}
```

---

## 14.3 Optional Properties vs Nullable Types

Understanding when to use optional properties (`?`) versus nullable types (`| null`) is important for API design.

### 14.3.1 Optional Properties

Optional properties use `?` and result in `undefined` when not present.

**Optional Property Patterns:**

```typescript
interface User {
  name: string;
  email?: string;           // Optional - undefined when missing
  phone?: string;           // Optional
  preferences?: {           // Optional nested object
    theme: "light" | "dark";
    notifications: boolean;
  };
}

// Valid objects
const user1: User = {
  name: "John"
  // email, phone, preferences are optional
};

const user2: User = {
  name: "Jane",
  email: "jane@example.com"
  // phone and preferences still optional
};

// Accessing optional properties
function getContact(user: User): string {
  // email is string | undefined
  if (user.email !== undefined) {
    return user.email;
  }
  
  if (user.phone !== undefined) {
    return user.phone;
  }
  
  return user.name;
}

// Optional chaining
function getTheme(user: User): string {
  // preferences might be undefined
  // theme might be undefined even if preferences exists
  return user.preferences?.theme ?? "light";
}
```

### 14.3.2 Nullable Types

Nullable types use `| null` and require explicit `null` assignment.

**Nullable Property Patterns:**

```typescript
interface Product {
  name: string;
  description: string | null;    // Must be present, can be null
  discontinuedDate: Date | null; // Explicitly null if not discontinued
  supplier: Supplier | null;     // Null if no supplier
}

interface Supplier {
  name: string;
}

// Must explicitly set null
const product: Product = {
  name: "Widget",
  description: null,        // Must include, even if null
  discontinuedDate: null,     // Must include
  supplier: null              // Must include
};

// Or provide values
const activeProduct: Product = {
  name: "Gadget",
  description: "A useful gadget",
  discontinuedDate: null,   // Still active
  supplier: { name: "Acme Corp" }
};

// Database/API mapping
interface DatabaseRow {
  id: number;
  deleted_at: string | null;  // SQL NULL maps to null
  updated_by: number | null; // Foreign key might be null
}

// Converting to optional (if needed)
type NullableToOptional<T> = {
  [K in keyof T]: T[K] extends null ? T[K] | undefined : T[K];
};
```

**When to Use Each:**

```
┌─────────────────────────────────────────────────────────────────────┐
│              Optional (?) vs Nullable (| null) Guide                 │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   Use Optional (?) when:                                            │
│   ✅ Property might not exist (input forms, partial updates)        │
│   ✅ JavaScript optional arguments                                  │
│   ✅ Configuration where omission means "use default"                 │
│   ✅ Internal APIs where undefined is conventional                  │
│                                                                     │
│   Use Nullable (| null) when:                                       │
│   ✅ Database fields that can be NULL                               │
│   ✅ API responses with explicit null values                        │
│   ✅ You need to distinguish "not set" from "set to null"           │
│   ✅ GraphQL responses (distinguishes null from undefined)          │
│                                                                     │
│   Conversion:                                                       │
│   ┌──────────────────────────────────────────────────────────────┐ │
│   │ // Optional to Nullable                                      │ │
│   │ type WithNullable<T> = {                                     │ │
│   │   [K in keyof T]: undefined extends T[K]                     │ │
│   │     ? Exclude<T[K], undefined> | null                      │ │
│   │     : T[K]                                                 │ │
│   │ };                                                          │ │
│   │                                                                │ │
│   │ // Nullable to Optional                                      │ │
│   │ type WithOptional<T> = {                                     │ │
│   │   [K in keyof T]: null extends T[K]                        │ │
│   │     ? Exclude<T[K], null> | undefined                    │ │
│   │     : T[K]                                                 │ │
│   │ };                                                          │ │
│   └──────────────────────────────────────────────────────────────┘ │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

---

## 14.4 Working with Nullable Values

TypeScript provides several operators and patterns for safely working with nullable values.

### 14.4.1 Optional Chaining (`?.`)

Optional chaining allows you to access properties or call methods that might be null or undefined without causing errors.

**Using Optional Chaining:**

```typescript
interface User {
  profile?: {
    address?: {
      city?: string;
      zipCode?: string;
    };
    contacts?: {
      email?: string;
      phone?: string;
    }[];
  };
}

const user: User = {
  profile: {
    address: {
      city: "New York"
      // zipCode is undefined
    },
    contacts: [
      { email: "john@example.com" }
      // phone is undefined
    ]
  }
};

const user2: User = {}; // profile is undefined

// Without optional chaining (verbose and error-prone)
function getCityUnsafe(user: User): string | undefined {
  if (user.profile !== undefined) {
    if (user.profile.address !== undefined) {
      return user.profile.address.city;
    }
  }
  return undefined;
}

// With optional chaining (clean and safe)
function getCitySafe(user: User): string | undefined {
  return user.profile?.address?.city;
  // Returns undefined if any part of chain is null/undefined
}

// Optional chaining with method calls
interface Logger {
  log?: (message: string) => void;
}

const logger: Logger = {};

// Safe method call
logger.log?.("message"); // Does nothing if log is undefined

// Optional chaining with function calls
function fetchData(callback?: (data: string) => void): void {
  const data = "fetched data";
  callback?.(data); // Only calls if callback exists
}

// Optional chaining with array access
const firstEmail = user.profile?.contacts?.[0]?.email;
// Returns undefined if contacts is undefined or empty

// Optional chaining with function returns
function getProcessor(): { process?: (x: number) => number } | null {
  return Math.random() > 0.5 ? { process: x => x * 2 } : null;
}

const result = getProcessor()?.process?.(5);
// Returns undefined if processor is null or process is undefined
```

### 14.4.2 Nullish Coalescing (`??`)

The nullish coalescing operator returns the right-hand operand when the left is `null` or `undefined` (but not `0`, `false`, or `""`).

**Using Nullish Coalescing:**

```typescript
// Difference between || and ??
const count = 0;
const empty = "";
const nulled = null;
const undef = undefined;

console.log(count || 10);    // 10 (0 is falsy)
console.log(count ?? 10);    // 0 (0 is not nullish)

console.log(empty || "default");  // "default" ("" is falsy)
console.log(empty ?? "default");  // "" ("" is not nullish)

console.log(nulled ?? "default");  // "default"
console.log(undef ?? "default");   // "default"

// Practical usage with configuration
interface Config {
  timeout?: number;
  retries?: number;
  enabled?: boolean;
}

function initialize(config: Config): Required<Config> {
  return {
    timeout: config.timeout ?? 5000,      // 0 is valid, don't default
    retries: config.retries ?? 3,         // 0 is valid, don't default
    enabled: config.enabled ?? true       // false is valid, don't default
  };
}

console.log(initialize({ timeout: 0 })); 
// { timeout: 0, retries: 3, enabled: true }

console.log(initialize({ timeout: 1000, retries: 0, enabled: false }));
// { timeout: 1000, retries: 0, enabled: false }

// Combining with optional chaining
interface User {
  settings?: {
    theme?: string;
    fontSize?: number;
  };
}

function getTheme(user: User): string {
  return user.settings?.theme ?? "light";
}

// Chaining nullish coalescing
const value = null ?? undefined ?? "fallback" ?? "another";
// "fallback" (first non-nullish)
```

### 14.4.3 Non-Null Assertion (`!`)

The non-null assertion operator tells TypeScript that a value is not null or undefined, even when type checking suggests otherwise.

**Using Non-Null Assertion:**

```typescript
function getElement(id: string): HTMLElement | null {
  return document.getElementById(id);
}

// Without non-null assertion
const element = getElement("app");
if (element === null) {
  throw new Error("Element not found");
}
element.innerHTML = "Hello"; // OK, narrowed by check

// With non-null assertion (use when you're certain)
const element2 = getElement("app")!;
// Tells TypeScript "trust me, it's not null"
element2.innerHTML = "Hello"; // OK

// Use cases for non-null assertion
interface User {
  name?: string;
}

const user: User = { name: "John" };

// When you know better than the compiler
console.log(user.name!.toUpperCase()); // Asserts name is defined

// After validation
function processUser(user: User | null) {
  if (user === null) {
    throw new Error("No user");
  }
  
  // TypeScript knows user is not null, but...
  const name = user.name!;
  // Use only if you're certain name exists despite being optional
}

// Array access when you're certain index exists
const items = ["a", "b", "c"];
const item = items[10]!; // Dangerous! Only use if certain

// Definite assignment in classes
class Container {
  value!: string; // Will be assigned later, trust me
  
  initialize() {
    this.value = "initialized";
  }
}
```

**Cautions with Non-Null Assertion:**

```typescript
// ❌ Bad: Using ! to suppress errors without validation
function bad(user: User | null) {
  console.log(user!.name!.toUpperCase()); // Might crash!
}

// ✅ Good: Proper validation
function good(user: User | null) {
  if (user?.name !== undefined) {
    console.log(user.name.toUpperCase());
  }
}

// ❌ Bad: Blind array access
const arr: number[] = [];
const val = arr[0]!; // Runtime error: undefined has no properties

// ✅ Good: Check bounds
if (arr.length > 0) {
  console.log(arr[0]);
}
```

---

## 14.5 Type Guards for Null Checks

You can create custom type guards specifically for null/undefined checking.

**Null Type Guards:**

```typescript
// Type guard for non-null values
function isDefined<T>(value: T | null | undefined): value is T {
  return value !== null && value !== undefined;
}

// Usage
const items: (string | null | undefined)[] = ["a", null, "b", undefined, "c"];

// Filter with type guard
const validItems = items.filter(isDefined);
// Type: string[] (null and undefined removed)

// Type guard for non-empty strings
function isNonEmptyString(value: string | null | undefined): value is string {
  return value !== null && value !== undefined && value.length > 0;
}

// Assert non-null helper
function assertDefined<T>(value: T | null | undefined): asserts value is T {
  if (value === null || value === undefined) {
    throw new Error("Value must be defined");
  }
}

function process(value: string | null) {
  assertDefined(value);
  // TypeScript knows value is string here
  console.log(value.toUpperCase());
}
```

---

## 14.6 Definite Assignment Assertion

TypeScript tracks variable initialization. The definite assignment assertion (`!`) tells TypeScript that a variable will definitely be assigned before use, even if TypeScript can't verify it.

**Using Definite Assignment:**

```typescript
class User {
  // Tell TypeScript these will be initialized
  id!: number;
  name!: string;
  
  constructor(data: { id?: number; name?: string }) {
    // Complex initialization logic
    if (data.id !== undefined) {
      this.id = data.id;
    } else {
      this.id = Math.random();
    }
    
    if (data.name !== undefined && data.name.length > 0) {
      this.name = data.name;
    } else {
      this.name = "Anonymous";
    }
  }
}

// Without assertion, we'd need to initialize in constructor
// or make them optional

// Variable definite assignment
let userId!: number;

function initialize() {
  userId = 123;
}

initialize();
console.log(userId); // OK - we told TypeScript it's assigned

// Class property with late initialization
class Database {
  connection!: Connection; // Will be set in connect()
  
  async connect(): Promise<void> {
    this.connection = await createConnection();
  }
  
  query(sql: string): void {
    // Must ensure connect() was called first
    this.connection.execute(sql);
  }
}
```

---

## 14.7 Chapter Summary and Exercises

### Chapter Summary

In this chapter, we covered nullable types comprehensively:

**Key Takeaways:**

1. **`null` vs `undefined`**:
   - `null`: Intentional absence of value
   - `undefined`: Not yet initialized or optional
   - Use `undefined` for optional, `null` for explicit absence

2. **Strict Null Checks**:
   - Enable `strictNullChecks` for safety
   - Forces explicit null handling
   - Catches potential runtime errors at compile time

3. **Optional vs Nullable**:
   - Optional (`?`): Property may not exist → `undefined`
   - Nullable (`| null`): Property exists but value is `null`

4. **Null-Safe Operators**:
   - Optional chaining (`?.`): Safe property/method access
   - Nullish coalescing (`??`): Default for null/undefined only
   - Non-null assertion (`!`): Tell TypeScript you know better (use sparingly)

5. **Best Practices**:
   - Always handle null cases explicitly
   - Use type guards for filtering
   - Prefer optional chaining over deep null checks
   - Avoid non-null assertion when possible

### Practical Exercises

**Exercise 1: Strict Null Checks**

Refactor unsafe code:

```typescript
// Given this unsafe code:
function processUser(user: User | null) {
  console.log(user.name.toUpperCase());
  console.log(user.email.toLowerCase());
}

// Refactor to:
// 1. Check if user is null and return early
// 2. Use optional chaining for email (might be undefined)
// 3. Provide default values using nullish coalescing
// 4. Add proper type guards

interface User {
  name: string;
  email?: string;
  phone: string | null;
}
```

**Exercise 2: Optional Chaining**

Build a safe config reader:

```typescript
// 1. Create a deeply nested Config interface with:
//    - database?: { host?: string; port?: number; credentials?: { username?: string; password?: string } }
//    - cache?: { enabled?: boolean; ttl?: number }
//    - features?: { newDashboard?: boolean; betaMode?: boolean }

// 2. Write a function getDatabaseHost(config: Config): string that:
//    - Returns config.database.host if all exist
//    - Returns "localhost" as default using nullish coalescing
//    - Uses optional chaining to safely navigate

// 3. Write a function isFeatureEnabled(config: Config, feature: string): boolean that:
//    - Safely checks if a feature flag exists and is true
//    - Returns false if path doesn't exist
```

**Exercise 3: Type Guards**

Create validation utilities:

```typescript
// 1. Create a type guard isNonNull<T>(value): value is T that:
//    - Filters out null and undefined from arrays
//    - TypeScript correctly infers the filtered type

// 2. Create a function parseConfig(json: string): Config | null that:
//    - Parses JSON safely
//    - Returns null if parsing fails
//    - Validates required fields exist

// 3. Create a function required<T>(value: T | null | undefined): T that:
//    - Throws error if value is null/undefined
//    - Uses assertion function to narrow type
//    - Can be used for "fail fast" validation

// 4. Use these to process an array of mixed values, filtering valid configs
```

**Exercise 4: Real-World API Handling**

Build a safe API client:

```typescript
// 1. Define types:
//    - ApiResponse<T> = { data: T | null; error: string | null; status: number }
//    - User = { id: number; name: string; email: string }

// 2. Create a function fetchUser(id: number): Promise<ApiResponse<User>> that:
//    - Simulates API call with random success/failure
//    - Returns proper ApiResponse structure

// 3. Create a function handleUserResponse(response: ApiResponse<User>): string that:
//    - Checks if error exists (using proper null checking)
//    - Checks if data exists
//    - Returns formatted user info or error message
//    - Uses exhaustive checking

// 4. Create a wrapper that throws on error (using assertion)
//    function unwrapUser(response: ApiResponse<User>): User
```

**Exercise 5: Null Safety Patterns**

Implement the Maybe monad pattern:

```typescript
// 1. Create a Maybe<T> type representing: T | null | undefined

// 2. Create utility functions:
//    - map<T, U>(maybe: Maybe<T>, fn: (val: T) => U): Maybe<U>
//    - flatMap<T, U>(maybe: Maybe<T>, fn: (val: T) => Maybe<U>): Maybe<U>
//    - filter<T>(maybe: Maybe<T>, predicate: (val: T) => boolean): Maybe<T>
//    - getOrElse<T>(maybe: Maybe<T>, defaultValue: T): T
//    - orElse<T>(maybe: Maybe<T>, alternative: Maybe<T>): Maybe<T>

// 3. Use these to safely process:
//    - Parse a number from string (might fail)
//    - Double it if even (filter)
//    - Convert to string with "Result: " prefix (map)
//    - Provide default "Invalid" if null (getOrElse)

// 4. Compare this approach to using optional chaining and nullish coalescing
```

### Additional Resources

- **TypeScript Handbook - Strict Null Checks**: https://www.typescriptlang.org/tsconfig#strictNullChecks
- **Optional Chaining**: https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Operators/Optional_chaining
- **Nullish Coalescing**: https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Operators/Nullish_coalescing
- **TypeScript Deep Dive - Null**: https://basarat.gitbook.io/typescript/recap/null-undefined

---

## Coming Up Next: Chapter 15 - Introduction to Generics

In the next chapter, we will explore:

- Understanding the problem generics solve
- Generic function syntax and type parameter inference
- Generic type variables and constraints
- Generic interfaces and classes
- Multiple type parameters
- Generic constraints with `extends`
- Default type parameters

Generics are one of TypeScript's most powerful features, enabling you to write reusable, type-safe code that works with multiple types while maintaining compile-time type checking.

---